In [1]:
import json

import mlflow
import numpy as np
import pandas as pd
import requests
from mlflow.tracking import MlflowClient

from train_rf import CATEGORICAL_FEATURES, NUMERICAL_FEATURES


In [2]:
current_experiment=dict(mlflow.get_experiment_by_name("fraud"))
experiment_id=current_experiment['experiment_id']

# Get this from UI or CLI
rf_parent_run = "9acecc61274b4dc7b81724bf4e84a89c"

## Find the best model

In [19]:
# To access MLFlow stuff we need to work with MlflowClient
client = MlflowClient()

# Searches runs for a specific attribute and filters them by Parent Run ID
runs = client.search_runs(
    [experiment_id], 
    f"tags.mlflow.parentRunId = '{rf_parent_run}'", 
    order_by=["metrics.test_PR_AUC DESC"]
)

# Select the best run according to test_PR_AUC metric
best_run = np.argmax([f.data.metrics['test_PR_AUC'] for f in runs])
best_pr_auc = np.round(runs[best_run].data.metrics['test_PR_AUC'], 4)

print(f"Experiment had {len(runs)} HP tuning round")
print(f"Best run - {best_run} with PR AUC of {best_pr_auc}")

Experiment had 10 HP tuning round
Best run - 2 with PR AUC of 0.1515


In [28]:
# log-model history is stored as string, so we need to "jsonify" it first
log_model_info = json.loads(runs[best_run].data.tags['mlflow.log-model.history'])[0]

# Construct a valid model URI
model_uri = 'runs:/' + log_model_info['run_id'] + '/' + log_model_info['artifact_path']
print(f"Best model URI - {model_uri}")


Best model URI - runs:/46596cc55a144ff49b7ccf81ad821d04/sklearn_models


## Load the best model

In [38]:
# Data sample to test the model
data = pd.read_csv("./data/raw/Base.csv", nrows=1)

In [39]:
# Load the model as pyfunc
sklearn_pyfunc = mlflow.pyfunc.load_model(model_uri=model_uri)
sklearn_pyfunc.predict(data)

array([[0.92940047, 0.07059953]])

## Register and Promote

In [40]:
model_name = 'fraud_RF'
model_version = 1

# Register model
mlflow.register_model(model_uri, model_name)

# Promote to Production
logs = client.transition_model_version_stage(name=model_name, version=model_version, stage="Production")

Successfully registered model 'fraud_RF'.
2023/02/12 13:20:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: fraud_RF, version 1
Created version '1' of model 'fraud_RF'.


## Load from Production Model Registry

In [44]:
model_name = 'fraud_RF'
stage = 'Production'

# Since it's a registered model in Production, we can load it like this now!
# No need for model URIs
model_registry_path = f'models:/{model_name}/{stage}'
production_model = mlflow.pyfunc.load_model(model_registry_path)

production_model.predict(data)

array([[0.92940047, 0.07059953]])

## Serve models

Run this command in the terminal: `mlflow models serve --model-uri models:/fraud_RF/Production -p 5001`

### Call from server

In [46]:
# Prepare the data to be sent to API
fraud_example = data[data['fraud_bool'] == 1][NUMERICAL_FEATURES + CATEGORICAL_FEATURES]
to_send = fraud_example.to_dict(orient='split')
to_send.pop("index", None)

[0]

In [47]:
# Prediction endpoint
url = 'http://127.0.0.1:5001/invocations'

# Preprocess the example
response = requests.post(url=url, data=json.dumps({"dataframe_split" :to_send}), headers={"Content-type": "application/json"})

# Load the response
response_json = json.loads(response.text)
print(response_json)

{'predictions': [[0.9294004729658535, 0.07059952703414646]]}
